In [2]:
import pandas as pd
import numpy as np
from pathlib import Path
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score, classification_report, confusion_matrix

In [3]:
df = pd.read_csv("./stations_cleaned_sample.csv")
originalColumns = df.columns.tolist()
df.shape

(1611, 21)

In [4]:
df.sample(10)

,station_code,monitoring_location,state_name,temp_min,temp_max,do_min,do_max,ph_min,ph_max,conductivity_min,...,bod_min,bod_max,nitrate_min,nitrate_max,fecal_coliform_min,fecal_coliform_max,total_coliform_min,total_coliform_max,fecal_streptococci_min,fecal_streptococci_max
100,3873,RIVER RAVI U/S STP AT BHARGA,HIMACHAL PRADESH,10.0,17.5,8.8,9.1,7.5,8.4,142.0,...,1.0,1.0,0.59,4.71,2.0,40.0,21.0,9200.0,2.0,2.0
527,1143,"RIVER TONS AT CHAKGHAT, M.P.",MADHYA PRADESH,17.3,29.8,6.7,7.8,7.6,8.2,236.0,...,1.5,1.9,0.45,1.16,2.0,2.0,46.0,49.0,2.0,2.0
893,5287,RIVER RANGPO AT D/S OF RORATHANG BRIDGE SIKKIM,NaN,13.0,22.0,7.9,8.7,6.7,7.3,32.0,...,1.6,2.4,1.80,4.50,29.0,123.0,52.0,173.0,NaN,NaN
695,1526,RIVER BRAHMPUTRA RIVER AT DHENUKHAPAHAR,ASSAM ASSAM,18.0,26.0,7.0,7.7,6.8,7.6,120.0,...,2.0,2.5,0.70,1.70,360.0,1100.0,910.0,2000.0,120.0,290.0
1575,1208,RIVER BHAVANI AT ELACHIVAZHY,PALAKKAD KERALA,23.0,30.0,7.1,8.3,7.6,8.1,80.0,...,1.0,2.4,0.30,0.46,2.0,400.0,100.0,800.0,2.0,60.0
1379,1167,RIVER BHIMA AT D/S OF BDG. NEAR YADGIR,KARNATAKA,26.0,34.0,7.0,8.0,8.1,8.5,390.0,...,2.0,3.0,0.35,11.56,33.0,490.0,280.0,2800.0,NaN,NaN
1130,3896,RIVER GANGUA NEAR RAJDHANI ENGINEERING COLLEGE,ODISHA,20.0,30.0,0.3,2.2,6.7,7.4,172.0,...,6.3,15.0,0.32,15.71,16000.0,160000.0,16000.0,160000.0,70.0,920.0
520,1363,RIVER RAPTI AFTER CONFL. OF R. HONIN NR. DOMIN...,UTTAR PRADESH,16.0,24.0,7.8,8.4,7.9,8.2,270.0,...,2.6,8.4,0.00,0.00,8000.0,16000.0,18000.0,30000.0,NaN,NaN
1273,1891,RIVER MANJEERA AT D/S INTAKE POINT TO BIDAR CITY,KARNATAKA,24.0,30.0,6.9,7.9,7.4,8.3,390.0,...,2.0,3.0,0.58,4.00,33.0,540.0,140.0,2800.0,NaN,NaN
833,4118,"RIVER TUIRIAL D/S, HEPP DAM, SAIPUM",KOLASIB DIST. MIZORAM,19.0,36.0,5.3,9.1,6.9,7.5,81.0,...,1.0,1.4,0.02,0.78,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
df.columns = [c.strip().lower().replace(" ", "_") for c in df.columns]

In [6]:
df.columns

Index(['station_code', 'monitoring_location', 'state_name', 'temp_min',
       'temp_max', 'do_min', 'do_max', 'ph_min', 'ph_max', 'conductivity_min',
       'conductivity_max', 'bod_min', 'bod_max', 'nitrate_min', 'nitrate_max',
       'fecal_coliform_min', 'fecal_coliform_max', 'total_coliform_min',
       'total_coliform_max', 'fecal_streptococci_min',
       'fecal_streptococci_max'],
      dtype='object')

In [7]:
turb_cols = [c for c in df.columns if "turbid" in c or c.endswith("ntu") or c == "turbidity"]
numeric_cols = df.select_dtypes(include=[np.number]).columns.tolist()

In [8]:
df.drop_duplicates(inplace=True)

In [9]:
df.isna().sum()

station_code                0
monitoring_location         6
state_name                124
temp_min                    7
temp_max                    8
do_min                      8
do_max                      8
ph_min                      8
ph_max                      8
conductivity_min            8
conductivity_max            8
bod_min                     8
bod_max                     8
nitrate_min                56
nitrate_max                56
fecal_coliform_min        184
fecal_coliform_max        185
total_coliform_min        224
total_coliform_max        224
fecal_streptococci_min    766
fecal_streptococci_max    767
dtype: int64

In [10]:
schema_report = pd.DataFrame({
    "column": df.columns,
    "dtype": [str(df[c].dtype) for c in df.columns],
    "non_null": [df[c].notna().sum() for c in df.columns],
    "nulls": [df[c].isna().sum() for c in df.columns],
    "unique": [df[c].nunique(dropna=True) for c in df.columns]
})
schema_report

,column,dtype,non_null,nulls,unique
0,station_code,int64,1605,0,1604
1,monitoring_location,object,1599,6,1598
2,state_name,object,1481,124,116
3,temp_min,float64,1598,7,149
4,temp_max,float64,1597,8,164
5,do_min,float64,1597,8,101
6,do_max,float64,1597,8,115
7,ph_min,float64,1597,8,77
8,ph_max,float64,1597,8,78
9,conductivity_min,float64,1597,8,569
